# Import Data - 2 
In this example we see the power of scripting over Excel in that, once we have code for one site, it's fairly easy to repeat the code for many sites.

In [ ]:
#Import libraries
import requests, io
import pandas as pd
import numpy as np
import ipyleaflet as lf

In [ ]:
#Generate a list of all sites in NC using the NWIS API 
# see https://waterservices.usgs.gov/rest/DV-Service.html
url = 'http://waterservices.usgs.gov/nwis/site'
params = {'stateCd':'NC',        #NC Sites Only
          'siteStatus':'all',    #Get all(active and inactive) sites
          'siteType':'ST',       #Stream sites only
          'parameterCd':'00060', #DISCHARGE(cfs), see https://help.waterdata.usgs.gov/codes-and-parameters/parameters
          'format':'rdb'         #RDB (csv)
         }

In [ ]:
#Send the requests and translate the response
response_raw = requests.get(url,params)

In [ ]:
#Decode to remove any odd characters (accents, etc.)
response_clean = response_raw.content.decode('utf-8')

In [ ]:
#Build a list of line numbers to skip from comments and data types
rowsToSkip= [] #Create an empty list to hold line numbers
lineNumber = 0 #Initialize a line number variable
#Iterate through lines, adding the line number to the list for comment lines
for lineNumber, lineString in enumerate(response_clean.split("\n")):
    if lineString.startswith('#'): 
        rowsToSkip.append(lineNumber)
    else:
        break
#Add another line 2 greater than the last
dataTypeLineNumber = rowsToSkip[-1] + 2
rowsToSkip.append(dataTypeLineNumber)

In [ ]:
#Convert the data into a data frame
df = pd.read_csv(io.StringIO(response_clean),
                 skiprows=rowsToSkip,     #Skip metadta and data spec lines
                 delimiter='\t',          #Set to tab delimited
                 dtype={'site_no':'str'}) #Set site_no to a string datatype
#Print the number of returned sites
df.head()

In [ ]:
#Rename the last two fields
df.rename(columns={'85235_00060_00003':'MeanFlow_cfs','85235_00060_00003_cd':'Confidence'},inplace=True)
df.head()

In [ ]:
df.dtypes

In [ ]:
#Save to a csv file
df.to_csv('GageData.csv',index=False)